In [16]:
#importing essential libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters 
num_epochs = 2
batch_size = 32
learning_rate = 0.01

# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a uniform size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='/kaggle/input/i-naturalist1/inaturalist_12K/train', transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

classes = dataset.classes
print(classes)

['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [30]:
#defining class for CNN
class FlexibleCNN(nn.Module):
    def __init__(self, num_classes, in_channels=3, conv_filters=[16, 32, 64, 128, 256], 
                 kernel_sizes=[3, 3, 3, 3, 3], pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=512, activation='ReLU'):
        self.conv_filters = conv_filters
        self.kernel_sizes = kernel_sizes
        self.pool_sizes = pool_sizes
        self.dense_units = dense_units
        #self.activation = activation
        
        super(FlexibleCNN, self).__init__()

        #convolutional layers
        self.conv_layers = nn.ModuleList()
        prev_out_size = 224
        in_channels = in_channels
        for out_channels, kernel_size, pool_size in zip(conv_filters, kernel_sizes, pool_sizes):
            self.conv_layers.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, kernel_size),
                    self.get_activation(activation),
                    nn.MaxPool2d(pool_size)
                )
            )
            in_channels = out_channels
            
        def cal_size(stride, padding, kernel_size, prev_size):
            new_size = (prev_size-kernel_size+2*padding)/stride + 1
            return new_size//2
        prev_size = 224
        
        for i in range(5):
            new_size1 = cal_size(stride = 1, padding = 0, kernel_size = self.kernel_sizes[i], prev_size= prev_size)
            prev_size = new_size1
        print(new_size1)

        #fully connected layers
        #self.fc1 = nn.Linear(conv_filters[-1] * 7 * 7, dense_units)
        self.fc1 = nn.Linear(self.conv_filters[4] * int(new_size1) * int(new_size1), self.dense_units)
        self.fc2 = nn.Linear(self.dense_units, 10)#num_classes)

    def forward(self, x):
        # Forward pass through convolutional layers
        for conv_layer in self.conv_layers:
            x = conv_layer(x)

        # Flatten the output for the fully connected layer
        x = torch.flatten(x, 1)

        # Forward pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def get_activation(self, activation):
        if activation == 'ReLU':
            return nn.ReLU()
        elif activation == 'LeakyReLU':
            return nn.LeakyReLU()
        elif activation == 'Sigmoid':
            return nn.Sigmoid()
        elif activation == 'Tanh':
            return nn.Tanh()
        else:
            raise ValueError("Invalid activation function")
'''
# Example usage:
model = FlexibleCNN(num_classes=10, 
                    conv_filters=[32, 64, 128, 256, 512], 
                    kernel_sizes=[3, 3, 3, 3, 3], 
                    pool_sizes=[2, 2, 2, 2, 2], 
                    dense_units=256, 
                    activation='ReLU')

#printing the model summary
print(model)'''

"\n# Example usage:\nmodel = FlexibleCNN(num_classes=10, \n                    conv_filters=[32, 64, 128, 256, 512], \n                    kernel_sizes=[3, 3, 3, 3, 3], \n                    pool_sizes=[2, 2, 2, 2, 2], \n                    dense_units=256, \n                    activation='ReLU')\n\n#printing the model summary\nprint(model)"

In [27]:
def get_activation(activation):
    if activation == 'ReLU':
        return nn.ReLU()
    elif activation == 'LeakyReLU':
        return nn.LeakyReLU()
    elif activation == 'Sigmoid':
        return nn.Sigmoid()
    elif activation == 'Tanh':
        return nn.Tanh()
    else:
        raise ValueError("Invalid activation function")

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Train the model
    model.train()
    for images, labels in train_loader:
#         images = images.to(device)
#         labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validate the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in val_loader:
#             images = images.to(device)
#             labels = labels.to(device)
            
            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            # Compute accuracy
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')

print('Finished Training')

Epoch [1/2], Validation Accuracy: 9.10%
Epoch [2/2], Validation Accuracy: 9.10%
Finished Training


In [18]:
!pip install wandb

In [20]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [21]:
wandb.login(key='cd7a6c2259e8886dc269bbf6f0f9e55089d3beeb')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [28]:
sweep_config = {
    'method': 'random',
    'name' : 'sweep test3',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3], [3,5,5,7,7], [7,7,5,5,3]]
        },
        'dropout': {
            'values': [0.2, 0.4]
        },
        'activation': {
            'values': ['ReLU']#, 'gelu', 'silu', 'mish']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': ['true','false']
        },
        'num_dense':{
            'values': [128, 256]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='Deep_Learning_Assignment2')

Create sweep with ID: o02hz50d
Sweep URL: https://wandb.ai/prabhat-kumar/Deep_Learning_Assignment2/sweeps/o02hz50d


In [31]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init(entity = 'prabhat-kumar') as run:

        run_name="-act_"+wandb.config.activation+"-ks"+str(wandb.config.kernel_size)+'-da'+wandb.config.data_augment+'-nd'+str(wandb.config.num_dense)
        wandb.run.name=run_name
        actv = get_activation(wandb.config.activation)
        model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org, 
                 kernel_sizes=wandb.config.kernel_size, pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=wandb.config.num_dense, activation=actv)
        
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
        for epoch in range(num_epochs):
    # Train the model
            model.train()
            for images, labels in train_loader:
            
#         images = images.to(device)
#         labels = labels.to(device)
        
        # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
        
        # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
    
    # Validate the model
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in val_loader:
                
#             images = images.to(device)
#             labels = labels.to(device)
            
            # Forward pass
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
            
            # Compute accuracy
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
        
                val_accuracy = 100 * correct / total
                print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')
                wandb.log({'validation_accuracy':val_accuracy})
    

        print('Finished Training')

        '''
        if wandb.config.optimizer == 'nesterov':
          model.train_nag(x_train, y_train, wandb.config.learning_rate, wandb.config.epochs, wandb.config.batch_size)
        if wandb.config.optimizer == 'momentum':
          model.train_momentum(x_train, y_train, wandb.config.learning_rate, wandb.config.epochs, wandb.config.batch_size)
        if wandb.config.optimizer == 'sgd':
          model.train_sgd(x_train, y_train, wandb.config.learning_rate, wandb.config.epochs, wandb.config.batch_size)
        if wandb.config.optimizer == 'adam':
          model.train_adam(x_train, y_train, wandb.config.learning_rate, wandb.config.epochs, wandb.config.batch_size)
        if wandb.config.optimizer == 'rmsprop':
          model.train_rmsprop(x_train, y_train, wandb.config.learning_rate, wandb.config.epochs, wandb.config.batch_size)
        if wandb.config.optimizer == 'nadam':
          model.train_nadam(x_train, y_train, wandb.config.learning_rate, wandb.config.epochs, wandb.config.batch_size)'''
        
        

wandb.agent(sweep_id, function=main,count=2) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: 8gkvahv4 with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: true
wandb: 	data_augment: false
wandb: 	dropout: 0.2
wandb: 	filt_org: [32, 32, 32, 32, 32]
wandb: 	kernel_size: [7, 7, 5, 5, 3]
wandb: 	num_dense: 256
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Traceback (most recent call last):
  File "/tmp/ipykernel_34/36126862.py", line 15, in main
    model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org,
  File "/tmp/ipykernel_34/1471933806.py", line 22, in __init__
    self.get_activation(activation),
  File "/tmp/ipykernel_34/1471933806.py", line 66, in get_activation
    raise ValueError("Invalid activation function")
ValueError: Invalid activation function


Run 8gkvahv4 errored:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_34/36126862.py", line 15, in main
    model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org,
  File "/tmp/ipykernel_34/1471933806.py", line 22, in __init__
    self.get_activation(activation),
  File "/tmp/ipykernel_34/1471933806.py", line 66, in get_activation
    raise ValueError("Invalid activation function")
ValueError: Invalid activation function

wandb: ERROR Run 8gkvahv4 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_34/36126862.py", line 15, in main
wandb: ERROR     model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org,
wandb: ERROR   F

Traceback (most recent call last):
  File "/tmp/ipykernel_34/36126862.py", line 15, in main
    model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org,
  File "/tmp/ipykernel_34/1471933806.py", line 22, in __init__
    self.get_activation(activation),
  File "/tmp/ipykernel_34/1471933806.py", line 66, in get_activation
    raise ValueError("Invalid activation function")
ValueError: Invalid activation function


Run b2hkgrg5 errored:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_34/36126862.py", line 15, in main
    model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org,
  File "/tmp/ipykernel_34/1471933806.py", line 22, in __init__
    self.get_activation(activation),
  File "/tmp/ipykernel_34/1471933806.py", line 66, in get_activation
    raise ValueError("Invalid activation function")
ValueError: Invalid activation function

wandb: ERROR Run b2hkgrg5 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "/tmp/ipykernel_34/36126862.py", line 15, in main
wandb: ERROR     model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org,
wandb: ERROR   F